# Zoom grader

Loads zoom reports from a pre-defined folder, and calculates attendance.

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [37]:
folder_name = '../../data/attendance/'

file_list = os.listdir(folder_name)
print(len(file_list))

44


In [70]:
df = pd.DataFrame()

for fname in file_list[:3]:
    # print(fname)
    fullname = folder_name + fname
    header = pd.read_csv(fullname, header=0, nrows=1)
    name = header['Topic'][0]
    datestring = header['Start Time'][0][:10]
    print(name, datestring)
    
    data = pd.read_csv(fullname, header=2)
    data['Date'] = datestring
    data['Meeting'] = name
    #print(data)
    
    df = df.append(data, ignore_index=True)

Bard Biosem 09/17/2020
Bard Biosem 09/24/2020
Bard Biosem 10/01/2020
